In [1]:
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from collections import Counter
from pymystem3 import Mystem
from classification_model import train_test_datasets, NeuralNetwork, train_nn, predict, multilabel_accuracy, get_test_loader
import re
from data_preprocessing import clean_data, apply_n_grams
import math
import joblib
import pickle
import numpy as np
from gensim.models import Word2Vec
import torch
import torch.nn as nn

In [2]:
career_habr = pd.read_json('/Users/alexandrasorokovikova/Data Science/НИР Сравнение IT/Comparison-of-IT-trends/data/career_habr.json')
career_habr

,time,name,skills
0,2015-07-28T10:14:09+03:00,Рекрутер ИТ,"[Другое, Подбор специалистов, Рекрутмент]"
1,2015-08-10T12:08:52+03:00,Full Stack Developer для match-3 проекта (acti...,"[Бэкенд, Бэкенд, Фронтенд, ActionScript, Starl..."
2,2015-07-28T14:26:53+03:00,Front-end разработчик (2ГИС для iOS),[]
3,2016-03-22T18:45:25+03:00,JavaScript разработчик / JS / Frontend developer,"[Фронтенд, JQuery, Backbone.js, React, Node.js..."
4,2015-07-28T16:46:50+03:00,Full stack web developer,"[Бэкенд, Бэкенд, Фронтенд, Python, Django]"
...,...,...,...
19169,2021-02-09T15:52:45+03:00,PHP-разработчик,"[Бэкенд, Средний (Middle), JavaScript, Git, ОО..."
19170,2021-02-09T16:54:59+03:00,Data Engineer,"[Бэкенд, Средний (Middle), Парсинг, SQL, MySQL..."
19171,2021-04-27T12:10:09+03:00,Senior Automation QA Engineer,"[Тестирование, Старший (Senior), JavaScript]"
19172,2021-02-09T17:11:00+03:00,Системный аналитик,"[Аналитика, Средний (Middle), SOAP, REST, XML,..."


In [3]:
fields = {0 : 'Железо, электроника',
         1 : 'Сетевые технологии, администрирование, devops',
         2 : 'Проектирование, тестирование',
         3 : 'Информационная безопасность',
         4 : 'Маркетинг, контекстная реклама',
         5 : 'Верстка, html, javascript',
         6 : 'Машинное обучение',
         7 : 'E-commerce, финансы',
         8 : 'Дизайн интерфейсов',
         9 : 'Бизнес, CRM',
         10 : 'SEO'}

In [4]:
word2vec_model = Word2Vec.load('habr_vocab_model_on_text')

## Подготовка признаков

In [5]:
def join(row):
    if len(row) != 0:
        return ' '.join(row)
career_habr['skills'] = career_habr['skills'].apply(join)
career_habr = career_habr.dropna()
career_habr.index = np.arange(len(career_habr))
career_habr.head()

,time,name,skills
0,2015-07-28T10:14:09+03:00,Рекрутер ИТ,Другое Подбор специалистов Рекрутмент
1,2015-08-10T12:08:52+03:00,Full Stack Developer для match-3 проекта (acti...,Бэкенд Бэкенд Фронтенд ActionScript Starling P...
2,2016-03-22T18:45:25+03:00,JavaScript разработчик / JS / Frontend developer,Фронтенд JQuery Backbone.js React Node.js Java...
3,2015-07-28T16:46:50+03:00,Full stack web developer,Бэкенд Бэкенд Фронтенд Python Django
4,2015-09-01T18:37:08+03:00,Разработчик Android,Бэкенд Приложения Тестирование Администрирован...


In [6]:
career_habr_train = career_habr['name'] + ' ' + career_habr['skills']
career_habr_train = pd.DataFrame(career_habr_train, columns = ['text'])
career_habr_train

,text
0,Рекрутер ИТ Другое Подбор специалистов Рекрутмент
1,Full Stack Developer для match-3 проекта (acti...
2,JavaScript разработчик / JS / Frontend develop...
3,Full stack web developer Бэкенд Бэкенд Фронтен...
4,Разработчик Android Бэкенд Приложения Тестиров...
...,...
19100,PHP-разработчик Бэкенд Средний (Middle) JavaSc...
19101,Data Engineer Бэкенд Средний (Middle) Парсинг ...
19102,Senior Automation QA Engineer Тестирование Ста...
19103,Системный аналитик Аналитика Средний (Middle) ...


In [7]:
career_habr_train['text'] = clean_data(career_habr_train['text'])
career_habr_train.head()

100%|██████████| 19105/19105 [00:10<00:00, 1889.31it/s]


,text
0,рекрутер ита подбор специалист рекрутмент
1,full stack developer match проект actionscript...
2,javascript разработчик js frontend developer ф...
3,full stack web developer бэкенд бэкенд фронтен...
4,разработчик android бэкенд приложение тестиров...


In [8]:
career_habr_train['text_n_grams'] = career_habr_train['text'].apply(apply_n_grams, word2vec_model=word2vec_model)
career_habr_train.head()

,text,text_n_grams
0,рекрутер ита подбор специалист рекрутмент,"[рекрутер, ита, подбор, специалист, рекрутмент]"
1,full stack developer match проект actionscript...,"[full, stack, developer, match, проект, action..."
2,javascript разработчик js frontend developer ф...,"[javascript, разработчик, js, frontend, develo..."
3,full stack web developer бэкенд бэкенд фронтен...,"[full, stack, web, developer, бэкенд, бэкенд, ..."
4,разработчик android бэкенд приложение тестиров...,"[разработчик, android, бэкенд, приложение, тес..."


In [9]:
def vectorize_text(text, word2vec_model):
    vector = [0]*100
    for word in text:
        try:
            vector += word2vec_model.wv[word]
        except:
            pass
    return vector

In [10]:
career_habr_train['vector'] = career_habr_train['text_n_grams'].apply(vectorize_text, word2vec_model=word2vec_model)
career_habr_train.head()

,text,text_n_grams,vector
0,рекрутер ита подбор специалист рекрутмент,"[рекрутер, ита, подбор, специалист, рекрутмент]","[0.7879882156848907, -2.5025654733181, 2.27332..."
1,full stack developer match проект actionscript...,"[full, stack, developer, match, проект, action...","[1.4451203867793083, -6.716171475127339, 4.145..."
2,javascript разработчик js frontend developer ф...,"[javascript, разработчик, js, frontend, develo...","[1.3440031316131353, -4.944431072100997, 3.431..."
3,full stack web developer бэкенд бэкенд фронтен...,"[full, stack, web, developer, бэкенд, бэкенд, ...","[1.4266254920512438, -2.4763052370399237, 4.92..."
4,разработчик android бэкенд приложение тестиров...,"[разработчик, android, бэкенд, приложение, тес...","[0.8636586079373956, -1.8647904694080353, 3.66..."


In [11]:
career_habr_train[list(fields.values())] = 0
career_habr_train.head()

,text,text_n_grams,vector,"Железо, электроника","Сетевые технологии, администрирование, devops","Проектирование, тестирование",Информационная безопасность,"Маркетинг, контекстная реклама","Верстка, html, javascript",Машинное обучение,"E-commerce, финансы",Дизайн интерфейсов,"Бизнес, CRM",SEO
0,рекрутер ита подбор специалист рекрутмент,"[рекрутер, ита, подбор, специалист, рекрутмент]","[0.7879882156848907, -2.5025654733181, 2.27332...",0,0,0,0,0,0,0,0,0,0,0
1,full stack developer match проект actionscript...,"[full, stack, developer, match, проект, action...","[1.4451203867793083, -6.716171475127339, 4.145...",0,0,0,0,0,0,0,0,0,0,0
2,javascript разработчик js frontend developer ф...,"[javascript, разработчик, js, frontend, develo...","[1.3440031316131353, -4.944431072100997, 3.431...",0,0,0,0,0,0,0,0,0,0,0
3,full stack web developer бэкенд бэкенд фронтен...,"[full, stack, web, developer, бэкенд, бэкенд, ...","[1.4266254920512438, -2.4763052370399237, 4.92...",0,0,0,0,0,0,0,0,0,0,0
4,разработчик android бэкенд приложение тестиров...,"[разработчик, android, бэкенд, приложение, тес...","[0.8636586079373956, -1.8647904694080353, 3.66...",0,0,0,0,0,0,0,0,0,0,0


## Поиск категорий

In [12]:
target_list = list(fields.values())
test_loader = get_test_loader(career_habr_train, target_list)

In [13]:
model = NeuralNetwork()
model.load_state_dict(torch.load("nn_classifier"))
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): Tanh()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Tanh()
    (4): Linear(in_features=512, out_features=11, bias=True)
  )
)

In [14]:
predictions = predict(model, test_loader)

In [15]:
def get_field(row):
    all_fields = []
    for i in row:
        all_fields.append(fields[i])
    return all_fields

career_habr['field'] = predictions
career_habr['field'] = career_habr['field'].apply(get_field)
career_habr

/Users/alexandrasorokovikova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/alexandrasorokovikova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,time,name,skills,field
0,2015-07-28T10:14:09+03:00,Рекрутер ИТ,Другое Подбор специалистов Рекрутмент,"[Бизнес, CRM, E-commerce, финансы, SEO]"
1,2015-08-10T12:08:52+03:00,Full Stack Developer для match-3 проекта (acti...,Бэкенд Бэкенд Фронтенд ActionScript Starling P...,"[Проектирование, тестирование, Верстка, html, ..."
2,2016-03-22T18:45:25+03:00,JavaScript разработчик / JS / Frontend developer,Фронтенд JQuery Backbone.js React Node.js Java...,"[Проектирование, тестирование, Верстка, html, ..."
3,2015-07-28T16:46:50+03:00,Full stack web developer,Бэкенд Бэкенд Фронтенд Python Django,"[Проектирование, тестирование, Сетевые техноло..."
4,2015-09-01T18:37:08+03:00,Разработчик Android,Бэкенд Приложения Тестирование Администрирован...,"[Сетевые технологии, администрирование, devops..."
...,...,...,...,...
19100,2021-02-09T15:52:45+03:00,PHP-разработчик,Бэкенд Средний (Middle) JavaScript Git ООП MyS...,"[Проектирование, тестирование, Железо, электро..."
19101,2021-02-09T16:54:59+03:00,Data Engineer,Бэкенд Средний (Middle) Парсинг SQL MySQL PHP ...,"[Проектирование, тестирование, Железо, электро..."
19102,2021-04-27T12:10:09+03:00,Senior Automation QA Engineer,Тестирование Старший (Senior) JavaScript,"[Верстка, html, javascript, E-commerce, финанс..."
19103,2021-02-09T17:11:00+03:00,Системный аналитик,Аналитика Средний (Middle) SOAP REST XML Разра...,"[Проектирование, тестирование, Железо, электро..."


In [17]:
career_habr.to_json('career_habr_final.json')